# Environment Setup

In [4]:
import nbformat

# Read the notebook (replace with your actual file name)
with open("5580372.ipynb") as f:
    nb = nbformat.read(f, as_version=4)

# Remove widgets metadata (fixes GitHub preview issue)
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Save a cleaned version of the notebook
with open("clean_notebook.ipynb", "w") as f:
    nbformat.write(nb, f)

print("✅ Cleaned notebook saved as clean_notebook.ipynb")

✅ Cleaned notebook saved as clean_notebook.ipynb


In [1]:
!pip install openai google-generativeai ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.4 MB/s eta 0:00:00


In [2]:
import openai
import google.generativeai as genai
import base64
import datetime
import pandas as pd
import re
import json
from dataclasses import dataclass, asdict
from IPython.display import display, clear_output
from ipywidgets import (
    FileUpload, Button, IntText, Dropdown, Text, VBox, HBox, ToggleButtons,
    Output, Password, HTML
)

In [3]:
try:
    from google.colab import output, files
    output.enable_custom_widget_manager()
    _COLAB = True
except Exception:
    files = None
    _COLAB = False

In [4]:
# Add OpenAI API Key
gemini_key_box = Text(description='Gemini Key:', placeholder='AIza...')
gpt_key_box = Text(description='GPT-4o Key:', placeholder='sk-...')

# Persona data model and description generator

In [5]:
from dataclasses import dataclass, field
from typing import List
import random

@dataclass
class Persona:
    age: int
    gender: str
    cultural_interests: str
    lifestyle: str
    social_values: str
    travel_and_exploration: str
    daily_routines: str
    visual_style: str
    emotional_aesthetics: str
    material_and_color_preferences: str
    personal_growth: str
    social_orientation: str
    aesthetic_orientation: str
    freedom_tendency: str
    identity_description: str
    expression_style: str

    def to_description(self) -> str:
        return (
            f"You are a {self.age}-year-old {self.gender.lower()} interested in {self.cultural_interests}. "
            f"You live a {self.lifestyle} lifestyle, value {self.social_values}, and enjoy {self.travel_and_exploration}. "
            f"Your daily routine includes {self.daily_routines}, and you’re drawn to {self.visual_style} visuals with {self.emotional_aesthetics} aesthetics. "
            f"You prefer {self.material_and_color_preferences}, focus on {self.personal_growth}, and care about {self.social_orientation}. "
            f"Your aesthetic is {self.aesthetic_orientation}, you believe in {self.freedom_tendency}, identify as a {self.identity_description}, "
            f"and express yourself through {self.expression_style}."
        )

## Fixed persona template

In [6]:
fixed_persona = Persona(
    age=23,
    gender="female",
    cultural_interests="Independent Cinema",
    lifestyle="Slow Living",
    social_values="Multiculturalism",
    travel_and_exploration="Urban Exploration",
    daily_routines="Note-Taking",
    visual_style="Vintage Film",
    emotional_aesthetics="Healing",
    material_and_color_preferences="Soft Textiles",
    personal_growth="Emotional Healing",
    social_orientation="Education Reform",
    aesthetic_orientation="Wabi-Sabi",
    freedom_tendency="Anti-Labeling",
    identity_description="Cultural Omnivore",
    expression_style="Storytelling"
)

## Diverse predefined personas

In [7]:
excel_path = "Persona_List.xlsx"
df = pd.read_excel(excel_path)

predefined_personas = [
    Persona(
        age=row["Age"],
        gender=row["Gender"],
        cultural_interests=row["Cultural Interests"],
        lifestyle=row["Lifestyle"],
        social_values=row["Social Values"],
        travel_and_exploration=row["Travel & Exploration"],
        daily_routines=row["Daily Routines"],
        visual_style=row["Visual Style"],
        emotional_aesthetics=row["Emotional Aesthetics"],
        material_and_color_preferences=row["Material & Color Preferences"],
        personal_growth=row["Personal Growth"],
        social_orientation=row["Social Orientation"],
        aesthetic_orientation=row["Aesthetic Orientation"],
        freedom_tendency=row["Freedom Tendency"],
        identity_description=row["Identity Description"],
        expression_style=row["Expression Style"]
    )
    for _, row in df.iterrows()
]

## Randomly generate personas

In [8]:
def generate_random_persona():
    return Persona(
        age=random.randint(18, 50),
        gender=random.choice(["female", "male", "non-binary"]),
        cultural_interests=random.choice([
            "Independent Cinema", "Philosophy", "Craft Revival", "Street Culture", "Subcultures",
            "Digital Art", "Traditional Painting", "Experimental Music"
        ]),
        lifestyle=random.choice([
            "Slow Living", "Night Owl", "Sustainable Living", "Digital Nomad",
            "Homebody", "Curated Consumption", "Nomadic Living", "Hyper-Productive"
        ]),
        social_values=random.choice([
            "Multiculturalism", "Environmentalism", "Liberalism", "Anti-Consumerism",
            "Tech Ethics", "Free Expression", "Equality", "Spiritual Freedom"
        ]),
        travel_and_exploration=random.choice([
            "Urban Exploration", "Volunteer Tourism", "Culinary Tourism", "Road Trip",
            "Backpacking", "Space Tourism", "Virtual Travel", "Solo Pilgrimage"
        ]),
        daily_routines=random.choice([
            "Note-Taking", "Multitasking", "Organized", "Instant Gratification",
            "Morning Rituals", "Digital Detox", "Flow Sessions", "Spontaneous Wandering"
        ]),
        visual_style=random.choice([
            "Vintage Film", "Monochrome", "Cyberpunk", "Nature-Inspired", "Futuristic",
            "Collage", "Minimalist", "Baroque Revival"
        ]),
        emotional_aesthetics=random.choice([
            "Healing", "Serene", "Absurd Humor", "Loneliness", "Passionate",
            "Nostalgia", "Melancholy", "Hopeful"
        ]),
        material_and_color_preferences=random.choice([
            "Soft Textiles", "Neon Brights", "Matte Finish", "Wood & Natural Materials",
            "Metallic & Industrial", "Earth Tones", "Transparent Acrylic", "Saturated Contrast"
        ]),
        personal_growth=random.choice([
            "Emotional Healing", "Creativity Cultivation", "Lifelong Learning",
            "Self-Expression", "Career Breakthrough", "Mindfulness Training"
        ]),
        social_orientation=random.choice([
            "Education Reform", "Aging Society", "Urbanization", "Climate Action",
            "Tech-Driven", "Decentralized Communities", "Post-Capitalism"
        ]),
        aesthetic_orientation=random.choice([
            "Wabi-Sabi", "Functionalism", "Ethnic Elements", "Maximalism",
            "Futurism", "Brutalism", "Retro Futurism"
        ]),
        freedom_tendency=random.choice([
            "Anti-Labeling", "Anti-Algorithm", "Bodily Autonomy", "Anti-Authority",
            "Free Speech", "Non-Conformity"
        ]),
        identity_description=random.choice([
            "Cultural Omnivore", "Observer", "Tradition Keeper", "Artist", "Innovator",
            "Border Crosser", "Symbol Hacker", "Postmodernist"
        ]),
        expression_style=random.choice([
            "Storytelling", "Logical & Structured", "Indirect & Subtle",
            "Ironic Humor", "Direct & Blunt", "Fragmented Collage"
        ])
    )

# Model Inference Logic

In [9]:
# Initialize the results table with column headers
if 'results' not in globals():
    results = pd.DataFrame(columns=[
        "timestamp", "image_name", "persona", "prompt", "response",
        "judgment_score", "confidence", "judgment_reason", "model"
    ])

In [10]:
# Initialize result table
results = pd.DataFrame(columns=[
    "timestamp", "image_name", "persona", "prompt", "response",
    "judgment_score", "confidence", "judgment_reason", "model"
])

In [11]:
# Encode image bytes to Base64 string
def encode_image_to_base64(file_bytes):
    return base64.b64encode(file_bytes).decode("utf-8")

In [12]:
def process_model_call(model_name):
    import base64, datetime, json, mimetypes

    # ------- PATCH: always get the current image robustly -------
    file_bytes, image_name, mime_type = None, None, None

    # 1) Prefer the unified getter if available (works with FileUpload & Choose Image)
    if 'get_current_image' in globals():
        b, n = get_current_image()
        if b:
            file_bytes = b
            image_name = n
            mime_type = mimetypes.guess_type(image_name)[0] or "image/jpeg"

    # 2) Fallback: read from uploader.value (handles dict / list / tuple)
    if file_bytes is None:
        v = getattr(uploader, "value", None)
        if v:
            if isinstance(v, dict) and len(v) > 0:
                item = list(v.values())[0]
                file_bytes = item.get("content")
                meta = item.get("metadata", {})
                image_name = meta.get("name") or "uploaded_image"
                mime_type  = meta.get("type") or mimetypes.guess_type(image_name)[0] or "image/jpeg"
            elif isinstance(v, (list, tuple)) and len(v) > 0:
                item = v[0]
                file_bytes = item.get("content")
                meta = item.get("metadata", {})
                image_name = item.get("name") or meta.get("name") or "uploaded_image"
                mime_type  = meta.get("type") or mimetypes.guess_type(image_name)[0] or "image/jpeg"

    # 3) Still no image -> exit
    if not file_bytes:
        print("Please upload an image first.")
        return

    # Save a temp copy for downstream use (optional but handy)
    tmp_path = f"/tmp/{image_name}"
    try:
        with open(tmp_path, "wb") as _f:
            _f.write(file_bytes)
        globals()['LATEST_IMAGE_BYTES'] = file_bytes
        globals()['LATEST_IMAGE_PATH']  = tmp_path
        globals()['LATEST_IMAGE_NAME']  = image_name
    except Exception:
        pass
    # ------- end PATCH -------

    # Build data URL for OpenAI image input
    base64_image = base64.b64encode(file_bytes).decode("utf-8")
    image_url = f"data:{mime_type};base64,{base64_image}"
    timestamp = datetime.datetime.now().isoformat(timespec='seconds')

    personas_to_use = []

    # Persona selection based on mode
    if persona_mode_selector.value == "Use Fixed Persona":
        personas_to_use = [fixed_persona]
    elif persona_mode_selector.value == "Use Diverse Persona":
        personas_to_use = predefined_personas
    elif persona_mode_selector.value == "Customize Persona":
        # Build a custom Persona object from the manual input widgets
        custom_persona = Persona(
            age=age_input.value,
            gender=gender_input.value,
            cultural_interests=cultural_interests_input.value,
            lifestyle=lifestyle_input.value,
            social_values=social_values_input.value,
            travel_and_exploration=travel_input.value,
            daily_routines=routine_input.value,
            visual_style=visual_style_input.value,
            emotional_aesthetics=emotional_input.value,
            material_and_color_preferences=material_input.value,
            personal_growth=growth_input.value,
            social_orientation=social_orientation_input.value,
            aesthetic_orientation=aesthetic_input.value,
            freedom_tendency=freedom_input.value,
            identity_description=identity_input.value,
            expression_style=expression_input.value
        )
        personas_to_use = [custom_persona]

    multi_persona_results = []

    for idx, persona_obj in enumerate(personas_to_use):
        persona = persona_obj.to_description()
        prompt = f"You are {persona}. Please describe your impression of this product."

        print(f"\n=== Running {model_name} for Persona {idx+1}: {persona} ===")

        try:
            if model_name == "GPT-4o":
                client = openai.OpenAI(api_key=gpt_key_box.value)
                response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                        {"role": "system", "content": "You are simulating a consumer giving feedback on product images."},
                        {"role": "user", "content": [
                            {"type": "text", "text": prompt},
                            {"type": "image_url", "image_url": {"url": image_url}}
                        ]}
                    ],
                    max_tokens=600
                )
                result_text = response.choices[0].message.content

            elif model_name == "Gemini":
                genai.configure(api_key=gemini_key_box.value)
                model = genai.GenerativeModel("gemini-1.5-pro")
                image_parts = [{"mime_type": mime_type, "data": file_bytes}]
                result = model.generate_content([prompt, image_parts[0]])
                # Some Gemini SDKs return None when text is empty; guard for that
                result_text = getattr(result, "text", "") or str(result)

            else:
                result_text = "(Unsupported model)"

            print(f"\nSimulated Feedback (Persona {idx+1}):\n", result_text)

            intermediate = {
                "timestamp": timestamp,
                "image_name": image_name,
                "persona": persona,
                "prompt": prompt,
                "response": result_text,
                "model": model_name
            }

            with open(f"last_result_{idx+1}.json", "w", encoding="utf-8") as f:
                json.dump(intermediate, f, ensure_ascii=False, indent=2)

            multi_persona_results.append(intermediate)

        except Exception as e:
            print(f"Error running {model_name} for Persona {idx+1}:", e)

    # Save all persona results
    with open("multi_results.json", "w", encoding="utf-8") as f:
        json.dump(multi_persona_results, f, ensure_ascii=False, indent=2)

# Jusge System

In [13]:
def evaluate_all_results():
    import os, re, json
    import pandas as pd

    # 读取多条模拟结果
    try:
        with open("multi_results.json", "r", encoding="utf-8") as f:
            results_data = json.load(f)
    except Exception as e:
        print("Error loading multi_results.json:", e)
        return None

    # Agent B：初始化 OpenAI（GPT）客户端
    client = openai.OpenAI(api_key=gpt_key_box.value)

    all_judgments = []

    for idx, item in enumerate(results_data):
        persona = item["persona"]
        response_text = item["response"]
        prompt = item["prompt"]
        image_name = item.get("image_name", "N/A")
        timestamp = item.get("timestamp", "")

        evaluation_prompt = f"""
You are a judgment agent evaluating whether a piece of product feedback sounds like it was written by a real human who aligns with a given persona.

Persona:
"{persona}"

Feedback:
"{response_text}"

Your task is to judge how human-like, natural, emotionally expressive, and persona-aligned this response is — including its tone, structure, personality, and voice.

—— Scoring Guidelines ——
90–100: Feels like a real person speaking in-character. Emotionally rich, casual, natural.
70–89: Mostly believable, with minor robotic moments.
50–69: Clearly AI-like but with some human effort.
30–49: Robotic, generic, emotionally flat.
0–29: No persona alignment, artificial tone.

Bonus Adjustments (+5 each):
+5: Uses casual, spoken, or internet-style language
+5: Has tone variation and mental flow — not stiff
+5: Expresses emotion
+5: Includes personal judgment or conflict
+5: Shows humor, sarcasm, or relatable commentary

Penalty Adjustments (-5 each):
-5: Structured summary or essay-like tone
-5: Too formal or stiff
-5: Lacks emotion or clear opinions
-5: Overly neutral or bland
-5: Encyclopedic or robotic explanation
-5: No human rhythm

Return JSON only:
{{
  "score": <number>,
  "confidence": <0-100>,
  "reasons": ["..."],
  "adjustments": ["..."]
}}
"""

        try:
            resp = client.chat.completions.create(
                model="gpt-4o",   # ✅ 使用旧版 gpt-4o
                messages=[
                    {"role": "system", "content": "You are a judgment agent evaluating persona alignment in product feedback."},
                    {"role": "user", "content": evaluation_prompt}
                ],
                max_tokens=500
            )

            raw_text = resp.choices[0].message.content.strip()
            m = re.search(r"```json\s*(\{.*?\})\s*```", raw_text, flags=re.DOTALL)
            cleaned = m.group(1) if m else raw_text
            structured = json.loads(cleaned)

            all_judgments.append({
                "timestamp": timestamp,
                "image_name": image_name,
                "persona": persona,
                "prompt": prompt,
                "response": response_text,
                "judgment_score": structured.get("score"),
                "confidence": structured.get("confidence"),
                "judgment_reason": " | ".join(structured.get("reasons", [])),
                "model": "gpt-4o"
            })

        except Exception as e:
            print(f"❌ Failed to evaluate Persona {idx+1}:", e)
            continue

    df = pd.DataFrame(all_judgments)
    if not df.empty:
        file_exists = os.path.exists("evaluated_multi_results.csv")
        df.to_csv("evaluated_multi_results.csv", mode="a", header=not file_exists, index=False)
    print("✅ All evaluations completed and saved to evaluated_multi_results.csv")
    return df

# UI Interface


In [16]:
# ================= Robust UI with reliable image input =================
import pandas as pd
from ipywidgets import (
    FileUpload, Button, ToggleButtons, Text, IntText, Dropdown,
    VBox, HBox, Output
)

# Enable widgets in Colab (safe no-op elsewhere)
try:
    from google.colab import output, files
    output.enable_custom_widget_manager()
    _COLAB = True
except Exception:
    files = None
    _COLAB = False

# Reuse API key inputs if already defined
try:
    gpt_key_box
except NameError:
    gpt_key_box = Text(description="GPT-4o Key:", layout={'width': '360px'})
try:
    gemini_key_box
except NameError:
    gemini_key_box = Text(description="Gemini Key:", layout={'width': '360px'})

# ---------------- Image input: FileUpload + Colab fallback ----------------
log_out = Output()  # central log area

uploader = FileUpload(accept='image/*', multiple=False)
choose_btn = Button(description="Choose Image") if _COLAB else None  # fallback picker

_current_image = {"bytes": None, "name": None}  # in-memory cache

def _clear_current_image():
    _current_image["bytes"] = None
    _current_image["name"]  = None

def clear_uploads(_=None):
    """Clear FileUpload and cache (fixes .clear() not working)."""
    try:
        uploader.value = ()
    except Exception:
        try:
            uploader.value = {}
        except Exception:
            pass
    if hasattr(uploader, "_counter"):
        try:
            uploader._counter = 0
        except Exception:
            pass
    _clear_current_image()
    with log_out:
        print("Uploader cleared. Ready for next upload.")

clear_button = Button(description="Clear Uploads")
clear_button.on_click(clear_uploads)

def _from_fileupload(upl, verbose_out=None):
    """Return (bytes, name) from FileUpload; supports dict/list; converts to bytes."""
    v = upl.value
    if not v:
        return None, None

    if isinstance(v, dict):
        if len(v) == 0:
            return None, None
        item = list(v.values())[-1]
        name = list(v.keys())[-1]
    elif isinstance(v, (tuple, list)):
        item = v[-1]
        name = item.get("name") or item.get("metadata", {}).get("name") or "uploaded_image"
    else:
        if verbose_out:
            with verbose_out:
                print(f"⚠️ Unrecognized FileUpload type: {type(v)}")
        return None, None

    content = item.get("content", None)
    if content is None:
        # selected but not committed by the gray 'Upload' button
        return None, None

    if isinstance(content, bytes):
        return content, name
    if hasattr(content, "tobytes"):
        return content.tobytes(), name
    try:
        return bytes(content), name
    except Exception:
        if verbose_out:
            with verbose_out:
                print(f"⚠️ Cannot convert content for {name}. type={type(content)}")
        return None, None

def handle_fileupload_change(change):
    b, n = _from_fileupload(uploader, verbose_out=log_out)
    if b:
        _current_image["bytes"] = b
        _current_image["name"]  = n
        with log_out:
            print(f"✅ File '{n}' uploaded via FileUpload ({len(b)} bytes).")
    else:
        with log_out:
            print("⚠️ File selected but not fully uploaded. "
                  "If FileUpload fails, use the 'Choose Image' button.")

uploader.observe(handle_fileupload_change, names="value")

def choose_image_via_colab(_=None):
    if not _COLAB:
        with log_out:
            print("⚠️ Colab file picker is unavailable in this environment.")
        return
    selected = files.upload()  # {name: bytes}
    if selected:
        name = next(iter(selected.keys()))
        b    = selected[name]
        _current_image["bytes"] = b
        _current_image["name"]  = name
        with log_out:
            print(f"✅ File '{name}' uploaded via files.upload ({len(b)} bytes).")

if choose_btn:
    choose_btn.on_click(choose_image_via_colab)

def get_current_image():
    """Unified getter: prefer FileUpload; else use cached Colab-upload image."""
    b, n = _from_fileupload(uploader, verbose_out=log_out)
    if b:
        _current_image["bytes"] = b
        _current_image["name"]  = n
        return b, n
    if _current_image["bytes"] is not None:
        return _current_image["bytes"], _current_image["name"]
    return None, None

# ---------------- Persona inputs (kept same as your Option A) ----------------
age_input = IntText(description="Age", value=23)
gender_input = Dropdown(description="Gender", options=["female", "male", "non-binary"], value="female")
cultural_interests_input = Text(description="Culture", value="Independent Cinema")
lifestyle_input = Text(description="Lifestyle", value="Slow Living")
social_values_input = Text(description="Values", value="Multiculturalism")
travel_input = Text(description="Travel", value="Urban Exploration")
routine_input = Text(description="Routine", value="Note-Taking")
visual_style_input = Text(description="Visual", value="Vintage Film")
emotional_input = Text(description="Emotion", value="Healing")
material_input = Text(description="Materials", value="Soft Textiles")
growth_input = Text(description="Growth", value="Emotional Healing")
social_orientation_input = Text(description="Society", value="Education Reform")
aesthetic_input = Text(description="Aesthetic", value="Wabi-Sabi")
freedom_input = Text(description="Freedom", value="Anti-Labeling")
identity_input = Text(description="Identity", value="Cultural Omnivore")
expression_input = Text(description="Expression", value="Storytelling")

custom_inputs_box = VBox([
    age_input, gender_input, cultural_interests_input, lifestyle_input, social_values_input,
    travel_input, routine_input, visual_style_input, emotional_input, material_input,
    growth_input, social_orientation_input, aesthetic_input, freedom_input,
    identity_input, expression_input
])

persona_mode_selector = ToggleButtons(
    options=["Use Fixed Persona", "Use Diverse Persona", "Customize Persona"],
    description="Persona Mode:"
)

input_control_output = Output()

def update_persona_mode(change):
    with input_control_output:
        input_control_output.clear_output()
        if change["new"] == "Customize Persona":
            display(custom_inputs_box)

persona_mode_selector.observe(update_persona_mode, names="value")
with input_control_output:
    input_control_output.clear_output()

# ---------------- Unified logging + safe wrapper ----------------
def _run_safely(fn, button=None, title=""):
    if button:
        button.disabled = True
    with log_out:
        from IPython.display import clear_output
        clear_output(wait=True)
        if title:
            print(title)
    try:
        fn()
    except Exception as e:
        import traceback
        with log_out:
            print("❌ Error:", e)
            traceback.print_exc()
    finally:
        if button:
            button.disabled = False

# ---------------- Buttons: Simulate / Evaluate / History ----------------
run_button_gemini = Button(description="Simulate")
evaluate_all_button = Button(description="Evaluate")
history_button = Button(description="Show Past Records")

def _on_simulate(_):
    def _job():
        with log_out:
            print(f"[DEBUG] Simulate clicked. uploader.value type={type(uploader.value)}, "
                  f"len={len(uploader.value) if hasattr(uploader.value,'__len__') else 'NA'}")

        # 1) Try to use whatever is already available
        img_bytes, img_name = get_current_image()

        # 2) If nothing, open picker (Colab) and cache the result
        if not img_bytes and files is not None:
            with log_out:
                print("⚠️ No image detected. Opening file picker...")
            selected = files.upload()  # blocks until you choose
            if selected:
                name = next(iter(selected.keys()))
                b    = selected[name]
                _current_image["bytes"] = b
                _current_image["name"]  = name
                img_bytes, img_name = b, name
                with log_out:
                    print(f"✅ File '{name}' uploaded via files.upload ({len(b)} bytes).")

        if not img_bytes:
            with log_out:
                print("⚠️ No image available. Please upload an image first.")
            return

        ran_ok = False
        try:
            # Your function: it now has the PATCH at the top to read current image
            process_model_call("Gemini")
            ran_ok = True
        finally:
            if ran_ok:
                clear_uploads()  # only clear after a real run

    _run_safely(_job, button=run_button_gemini, title="▶ Running Simulation…")

run_button_gemini.on_click(_on_simulate)

def evaluate_and_show(_=None):
    try:
        df = evaluate_all_results()
    except NameError:
        with log_out:
            print("⚠️ evaluate_all_results() is not defined. Run its definition cell first.")
        return
    if df is None or df.empty:
        with log_out:
            print("No evaluations to display.")
        return
    display(df[["response", "judgment_score", "confidence", "judgment_reason"]])

def _on_evaluate(_):
    _run_safely(lambda: evaluate_and_show(), button=evaluate_all_button, title="▶ Running Evaluation…")

evaluate_all_button.on_click(_on_evaluate)

def show_history(_):
    def _job():
        try:
            df = pd.read_csv("evaluated_multi_results.csv")
            display(df)
        except Exception as e:
            with log_out:
                print("⚠️ Unable to read history file:", e)
    _run_safely(_job, button=history_button, title="▶ Loading History…")

history_button.on_click(show_history)

# ---------------- Layout ----------------
top_inputs = [gpt_key_box, gemini_key_box]
upload_row = [uploader, clear_button]
if choose_btn:
    upload_row.append(choose_btn)

display(VBox([
    HBox(top_inputs),
    HBox(upload_row),
    persona_mode_selector, input_control_output,
    HBox([run_button_gemini, evaluate_all_button, history_button]),
    log_out
]))
# ======================================================================